# Regular Expressions in Pandas, SQL, and NLP

In this section we will learn a few practical places we can apply regular expressions through Python libraries. Regular expressions are supported in many, many places but hopefully this will give an idea of how regular expressions can be used for common libraries. 

## Pandas 

When you import a CSV, you typically would use Pandas in a Python environment. 

In [ ]:
import pandas as pd 

url = r"https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/classification/iris.csv"
df = pd.read_csv(url)
df

Recall in the last section how we manually separated only the `species` column from the rest of the data. We can achieve this using the `sep` argument and provide a regular expression. We will need to tell Pandas to use the `python` engine to handle the regular expression. 

In [ ]:
import re 

pd.read_csv(url, sep=",(?=[a-z]+$)", engine='python')

Going back to our original DataFrame with columns predictably separated, let's say we wanted to match a regular expression against a field. We can use the `str.match()` function to return a `Boolean` array of values, and then qualify only those records. Below we match only species that start with a `v` and the third character is an `r`, as specified by the regex `^v[a-z]r.*`.  

In [ ]:
df[df['species'].str.match("^v[a-z]r.*") == True]

Sure enough we get records where the species are `versicolor` and `virginica`. 

This example may be slightly contrived, but we can also replace a regular expression pattern with different text. Below we take that regex pattern and replace those three latters with "XXX". This could be helpful if you are trying to replace sensitive information like social security numbers. 

In [ ]:
df['species'].str.replace("^v[a-z]r", "XXX", regex=True)

There are a lot of places that accept regular expressions in Pandas, so be sure to keep an eye out for regex-related parameters in the functions you use!

## SQL 

Another place you can leverage regular expressions is most mainstream SQL platforms like MySQL, PostgreSQL, Microsoft SQL Server, Oracle, and SQLite. Let's try it out on SQLite just to see this in action. Note that each SQL platform may implement regular expression function calls differently, just like Python and Java will have their own functions to match, replace, and split text using regular expressions. The regular expressions are largely the same across all platforms, but how you pass them via functions and operators will vary. 

Let's download and open a SQLite database, and query a `CUSTOMER` table. For good measure, we'll use Pandas to conveniently display the results in a `DataFrame`. 

In [ ]:
import urllib.request
import sqlite3
import pandas as pd 

urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')


sql = "SELECT * FROM CUSTOMER"

pd.read_sql(sql, conn)

While other SQL platforms are often ready-to-go to use regular expressions, we have to enable them with SQLite. To use a regular expression to match records on a given field, we need to implement the `REGEXP` function. Thankfully we can do this by simply passing a Python function to the SQLite connection by this name, and we are set.

In [ ]:
import re 

def regexp(pattern, string):
    return 1 if re.search(pattern, string) else 0

conn.create_function('regexp', 2, regexp)

Below we query records where the `ADDRESS` ends in `Dr` or `Ave`, which we capture with a regular expression. 

In [ ]:
sql = "SELECT * FROM CUSTOMER WHERE ADDRESS REGEXP '.*(Dr|Ave)$'"

pd.read_sql(sql, conn)

While it will vary by SQL platform, you can expect to find functions to split and replace text using regular expression patterns as well. 

# NLP using spaCy 

If you have ever dabbled in natural language processing (NLP), or explored how to build large language models, you will likely know that tokenization is a fundamental step to take text data and turn it into numeric data. 

Before we get started, let's get spaCy installed. 

In [ ]:
!pip install spacy 
!python -m spacy download en_core_web_sm

When you tokenize text data, you typically will tokenize words, names, and other dictionary-esque strings. But sometimes you may want to tokenize IP addresses, phone numbers, and other structurally-formatted values and perform custom matching on those. Maybe you want to match different variants or spellings of the same word, like favorite or favourite. 

We can use regular expresssions to aid spaCy in identifying phone numbers like this.

In [ ]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")
doc = nlp("My phone number is 742-278-0572")

expression = r"[0-9]{3}-[0-9]{3}-[0-9]{4}"
for match in re.finditer(expression, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    if span is not None:
        print("Found match:", span.text)

If you are familiar with spaCy, definitely check out the [documentation on rule-based matching](https://spacy.io/usage/rule-based-matching). 

## Exercise

Find a way to modify the `DataFrame` below to filter only for records where the `STREET_ADDRESS` field has a 3-digit street number. 

In [ ]:
import urllib.request
import sqlite3
import pandas as pd 

urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

df = pd.read_sql("SELECT * FROM CUSTOMER", conn)

### SCROLL DOWN FOR ANSWER
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

You can achieve this task in two ways, but both use the regex `^[0-9]{3}\s`. 

Modify the SQL query to use regular expressions: 

In [ ]:
import urllib.request
import sqlite3
import pandas as pd 

urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

import re 

def regexp(pattern, string):
    return 1 if re.search(pattern, string) else 0

conn.create_function('regexp', 2, regexp)

df = pd.read_sql(r"SELECT * FROM CUSTOMER WHERE ADDRESS REGEXP '^[0-9]{3}\s'", conn)
df

Filter the dataframe using a regular expression: 

In [ ]:
import urllib.request
import sqlite3
import pandas as pd 

urllib.request.urlretrieve("https://github.com/thomasnield/anaconda_intro_to_sql/blob/main/company_operations.db?raw=true", "company_operations.db")
conn = sqlite3.connect('company_operations.db')

df[df["ADDRESS"].str.match(r"^[0-9]{3}\s")]